In [1]:
import json

### 1. create the structure of the dataset (context_qa type)

In [2]:
def get_dictonary(c,q):
    dictionary = {
        "context":"table("+c+")",
        "question":q['question'],
        "answer":q['answer'],
    }
    return dictionary

### 2. Synthetic data
A simple example: to automate the process, you should randomly generate object poses.

In [ ]:
import random

def generate_dataset_and_qa():
    # Define object names
    fruit = ["apple", "banana", "pear", "melon"]
    recipient = ["cup", "bowl", "box", "basket"]
    object_names = fruit + recipient

    # Initialize a list to store the final questions and answers
    questions_and_answers = []

    # Generate dataset with object positions and process them in a single loop
    for _ in range(1000000):  # Repeat 1mln times
        # Generate positions for each object
        objects = [
            {
                "name": name,
                "x": random.randint(-500, 500),
                "y": random.randint(-500, 500),
                "z": random.randint(0, 500),
            }
            for name in object_names
        ]

        # Generate the context for this set of objects
        context = "; ".join(f"{obj['name']} x: {obj['x']}, y: {obj['y']}, z: {obj['z']}" for obj in objects) + ";"

        # Generate two types of questions and answers for each set of objects
        obj1 = random.choice(objects)
        obj2 = random.choice(objects)
        
        # Ensure obj1 and obj2 are not the same and obj2 is not a recipient
        while obj2['name'] in recipient or obj1 == obj2:
            obj2 = random.choice(objects)

        x_a, y_a, z_a = obj1['x'], obj1['y'], obj1['z']
        x_c, y_c, z_c = obj2['x'], obj2['y'], obj2['z']

        # Question 1: Pick object O1 and place it next to O2
        q1 = f"Pick the object {obj1['name']} and place next to the {obj2['name']}."
        a1 = (
            f"go to x: {x_a}, y: {y_a}, z: {z_a}+30; "
            f"go to x: {x_a}, y: {y_a}, z: {z_a}; "
            f"close the gripper; "
            f"go to x: {x_c}+10, y: {y_c}, z: {z_c}+10; "
            f"open the gripper; "
            "go home;"
        )
        if obj1['name'] in recipient:
            a1 = f"This task cannot be performed as {obj1['name']} is a recipient."

        # Ensure obj2 is not a fruit for Question 2
        while obj2['name'] in fruit or obj1 == obj2:
            obj2 = random.choice(objects)
            x_c, y_c, z_c = obj2['x'], obj2['y'], obj2['z']

        # Question 2: Pick object O1 and place it in O2
        q2 = f"Pick the object {obj1['name']} and place it in the {obj2['name']}."
        a2 = (
            f"go to x: {x_a}, y: {y_a}, z: {z_a}+30; "
            f"go to x: {x_a}, y: {y_a}, z: {z_a}; "
            f"close the gripper; "
            f"go to x: {x_c}, y: {y_c}, z: {z_c}+10; "
            f"open the gripper; "
            "go home;"
        )

        q, a = random.choice([(q1, a1), (q2, a2)])
        qa = get_dictonary(context, {"question": q, "answer": a})

        questions_and_answers.append(qa)

    return questions_and_answers

# Generate the dataset and question-answer pairs
final_data = generate_dataset_and_qa()

# Example: Print the first entry
print(final_data)

KeyboardInterrupt: 

In [ ]:
# save in json file
with open('data.json', 'w') as f:
    json.dump(final_data, f, indent=4)


# Load the json file
with open('data.json', 'r') as f:
    data = json.load(f)

### 5. Login to HuggingFace

In [ ]:
access_token = "secret"
from huggingface_hub import login
login(token=access_token)

/miniconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### 6. Upload the dataset to HuggingFace

In [12]:
import datasets
from datasets import load_dataset
dataset = load_dataset('json', data_files='train.json')
dataset.push_to_hub("marcomaccarini/test_lab")

Generating train split: 1 examples [00:00, 113.42 examples/s]
